### 캐싱(Chacing)

- 동일한 완료를 여러번 요청 하는 경우 LLM 공급자에 대한 API 호출 횟수를 줄여 비용 절감 도움.
- LLM 제공업체에 대한 API 호출 횟수를 줄여 애플리케이션 속도 증가에 도움.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("Cache")

LangSmith 추적을 시작합니다.
[프로젝트명]
Cache


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate


llm = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=0
)

prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘.")

chain = prompt | llm

In [6]:
%%time
response = chain.invoke({"country":"한국"})
print(response)

content='한국은 동아시아에 위치한 한반도의 국가로, 남한과 북한으로 나뉘어 있습니다. 서울은 수도이며, 경제, 문화, 기술의 중심지입니다. 한국은 K-팝, 드라마, 음식 등으로 세계적으로 유명하며, 전통과 현대가 조화를 이루는 독특한 문화를 가지고 있습니다. 또한, 교육 수준이 높고, IT 산업이 발달해 있습니다.' response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 21, 'total_tokens': 113}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b0abffe81', 'finish_reason': 'stop', 'logprobs': None} id='run-0307efb5-f0b5-4049-a96c-6b383c900886-0' usage_metadata={'input_tokens': 21, 'output_tokens': 92, 'total_tokens': 113}
CPU times: user 1.36 s, sys: 625 ms, total: 1.99 s
Wall time: 3.08 s


### InMemoryCache

인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환<br>
반복되는 요청에도 비용 발생 X. 프로그램이 꺼지면 Cache 휘발.


사용자별 Cache를 저장해야 할 때 InMemoryCache 유용.

In [8]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
response = chain.invoke({"country":"한국"})
print(response.content)

한국은 동아시아에 위치한 한반도의 국가로, 남한과 북한으로 나뉘어 있습니다. 서울은 수도이며, 경제, 문화, 기술의 중심지입니다. 한국은 K-팝, 드라마, 음식 등으로 세계적으로 유명하며, 전통과 현대가 조화를 이루는 독특한 문화를 가지고 있습니다. 또한, 교육 수준이 높고, IT 산업이 발달해 있습니다.
CPU times: user 15.9 ms, sys: 2.93 ms, total: 18.8 ms
Wall time: 3.15 s


In [9]:
%%time
response = chain.invoke({"country":"한국"})
print(response.content)

한국은 동아시아에 위치한 한반도의 국가로, 남한과 북한으로 나뉘어 있습니다. 서울은 수도이며, 경제, 문화, 기술의 중심지입니다. 한국은 K-팝, 드라마, 음식 등으로 세계적으로 유명하며, 전통과 현대가 조화를 이루는 독특한 문화를 가지고 있습니다. 또한, 교육 수준이 높고, IT 산업이 발달해 있습니다.
CPU times: user 4.04 ms, sys: 582 μs, total: 4.62 ms
Wall time: 4.48 ms


### SQLite Cache

프로그램을 종료 후 다시 시작해도 Cache 유지 가능.

In [12]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

if not os.path.exists("cache"):
    os.makedirs("cache")
set_llm_cache(SQLiteCache("cache/langchain.db"))

In [14]:
%%time

response = chain.invoke({"country":"한국"})
print(response.content)

한국은 동아시아에 위치한 한반도의 국가로, 남한과 북한으로 나뉘어 있습니다. 서울은 수도이며, 경제, 문화, 기술의 중심지입니다. 한국은 K-팝, 드라마, 음식 등으로 세계적으로 유명하며, 전통과 현대가 조화를 이루는 독특한 문화를 가지고 있습니다. 또한, 교육 수준이 높고, IT 산업이 발달해 있습니다.
CPU times: user 7.42 ms, sys: 2.45 ms, total: 9.87 ms
Wall time: 8.42 ms
